## Building Your First Agent with AutoGen

<a target="_blank" href="https://colab.research.google.com/github/victordibia/multiagent-systems-with-autogen/blob/main/ch02/ch2_1_first_agent.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>



![Multi-agent](https://github.com/victordibia/multiagent-systems-with-autogen/blob/main/docs/images/agent.png?raw=true)

Multi-agent AI apps (a setup where multiple agents powered by GenAI models collaborate autonomously) hold promise in addressing complex tasks. In this notebook, the goal is singular - help you build your Agent with AutoGen.



## Installation

We will be using the AutoGen high level API (AgentChat),  we will need to install the package accordingly. 

```bash
pip install -U "autogen-agentchat" "autogen-ext[openai,azure]"
```

In [1]:
# !pip install -U "autogen-agentchat" "autogen-ext[openai,azure]"

## Defining a Generative AI Model for Your Agent

Agents are powered by Generative AI models. We will be using an OpenAI model in this example.  You can create your own API key at [OpenAI][(https://platform.openai.com/signup](https://platform.openai.com/).

OpenAI models require an API key. It is recommended that we set this as an environment variable.  

- create a new file called `.env` in the folder for this notebook.
- add the following line to the file, replacing `YOUR_OPENAI_API_KEY` with your actual OpenAI API key:

```bash
OPENAI_API_KEY=YOUR_OPENAI_API_KEY
```
- save the file.


## Defining a Basic Agent  



In [2]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    # api_key="YOUR_API_KEY", add if not using .env
)

agent = AssistantAgent(
    name="weather_agent",
    model_client=model_client, 
    system_message="You are a helpful assistant.",  
)

await Console(agent.run_stream(task="What is the weather in New York?")) 
await model_client.close()

---------- TextMessage (user) ----------
What is the weather in New York?
---------- TextMessage (weather_agent) ----------
I'm unable to provide real-time weather updates or current data. For the most accurate and current weather information in New York, I recommend checking a reliable weather website or app, such as the National Weather Service, Weather.com, or a similar service.


## Giving Your Agent Access to A Tool 


In [3]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Define a model client. You can use other model client that implements
# the `ChatCompletionClient` interface.
model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    # api_key="YOUR_API_KEY",
)


# Define a simple function tool that the agent can use.
# For this example, we use a fake weather tool for demonstration purposes.
async def get_weather(city: str) -> str:
    """Get the weather for a given city."""
    return f"The weather in {city} is 73 degrees and Sunny."


# Define an AssistantAgent with the model, tool, system message, and reflection enabled.
# The system message instructs the agent via natural language.
agent = AssistantAgent(
    name="weather_agent",
    model_client=model_client,
    tools=[get_weather],
    system_message="You are a helpful assistant.",
    reflect_on_tool_use=True,
    model_client_stream=True,  # Enable streaming tokens from the model client.
)


# Run the agent and stream the messages to the console.
async def main() -> None:
    await Console(agent.run_stream(task="What is the weather in New York?"))
    # Close the connection to the model client.
    await model_client.close()


# NOTE: if running this inside a Python script you'll need to use asyncio.run(main()).
await main()


---------- TextMessage (user) ----------
What is the weather in New York?
---------- ToolCallRequestEvent (weather_agent) ----------
[FunctionCall(id='call_WG6i3LtD5VlOQWgvVyedpt3A', arguments='{"city":"New York"}', name='get_weather')]
---------- ToolCallExecutionEvent (weather_agent) ----------
[FunctionExecutionResult(content='The weather in New York is 73 degrees and Sunny.', name='get_weather', call_id='call_WG6i3LtD5VlOQWgvVyedpt3A', is_error=False)]
---------- ModelClientStreamingChunkEvent (weather_agent) ----------
The weather in New York is currently 73 degrees and sunny.
